<a href="https://colab.research.google.com/github/khaledrefai/gensim_arabic/blob/master/gensim_arabic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget -O full_grams_cbow_300_twitter https://archive.org/download/full_grams_cbow_300_twitter/full_grams_cbow_300_twitter.zip

--2019-03-09 14:05:20--  https://archive.org/download/full_grams_cbow_300_twitter/full_grams_cbow_300_twitter.zip
Resolving archive.org (archive.org)... 207.241.224.2
Connecting to archive.org (archive.org)|207.241.224.2|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://ia802903.us.archive.org/26/items/full_grams_cbow_300_twitter/full_grams_cbow_300_twitter.zip [following]
--2019-03-09 14:05:26--  https://ia802903.us.archive.org/26/items/full_grams_cbow_300_twitter/full_grams_cbow_300_twitter.zip
Resolving ia802903.us.archive.org (ia802903.us.archive.org)... 207.241.233.53
Connecting to ia802903.us.archive.org (ia802903.us.archive.org)|207.241.233.53|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3325529808 (3.1G) [application/zip]
Saving to: ‘full_grams_cbow_300_twitter’

full_grams_cbow_300 100%[===================>]   3.10G   892KB/s    in 38m 39s 

2019-03-09 14:44:06 (1.37 MB/s) - ‘full_grams_cbow_300_twitter’ saved [

In [9]:
!unzip full_grams_cbow_300_twitter

Archive:  full_grams_cbow_300_twitter
  inflating: full_grams_cbow_300_twitter.mdl  
  inflating: full_grams_cbow_300_twitter.mdl.trainables.syn1neg.npy  
  inflating: full_grams_cbow_300_twitter.mdl.wv.vectors.npy  


In [0]:
import gensim
import re
import numpy as np
from nltk import ngrams

In [0]:
# Clean/Normalize Arabic Text
def clean_str(text):
    search = ["أ","إ","آ","ة","_","-","/",".","،"," و "," يا ",'"',"ـ","'","ى","\\",'\n', '\t','&quot;','?','؟','!']
    replace = ["ا","ا","ا","ه"," "," ","","",""," و"," يا","","","","ي","",' ', ' ',' ',' ? ',' ؟ ',' ! ']
        
    #remove tashkeel
    p_tashkeel = re.compile(r'[\u0617-\u061A\u064B-\u0652]')
    text = re.sub(p_tashkeel,"", text)
    
    #remove longation
    p_longation = re.compile(r'(.)\1+')
    subst = r"\1\1"
    text = re.sub(p_longation, subst, text)
    
    text = text.replace('وو', 'و')
    text = text.replace('يي', 'ي')
    text = text.replace('اا', 'ا')
    
    for i in range(0, len(search)):
        text = text.replace(search[i], replace[i])
    
    #trim    
    text = text.strip()

    return text
def get_vec(n_model,dim, token):
    vec = np.zeros(dim)
    is_vec = False
    if token not in n_model.wv:
        _count = 0
        is_vec = True
        for w in token.split("_"):
            if w in n_model.wv:
                _count += 1
                vec += n_model.wv[w]
        if _count > 0:
            vec = vec / _count
    else:
        vec = n_model.wv[token]
    return vec

def calc_vec(pos_tokens, neg_tokens, n_model, dim):
    vec = np.zeros(dim)
    for p in pos_tokens:
        vec += get_vec(n_model,dim,p)
    for n in neg_tokens:
        vec -= get_vec(n_model,dim,n)
    
    return vec 
## -- Retrieve all ngrams for a text in between a specific range
def get_all_ngrams(text, nrange=3):
    text = re.sub(r'[\,\.\;\(\)\[\]\_\+\#\@\!\?\؟\^]', ' ', text)
    tokens = [token for token in text.split(" ") if token.strip() != ""]
    ngs = []
    for n in range(2,nrange+1):
        ngs += [ng for ng in ngrams(tokens, n)]
    return ["_".join(ng) for ng in ngs if len(ng)>0 ]

## -- Retrieve all ngrams for a text in a specific n
def get_ngrams(text, n=2):
    text = re.sub(r'[\,\.\;\(\)\[\]\_\+\#\@\!\?\؟\^]', ' ', text)
    tokens = [token for token in text.split(" ") if token.strip() != ""]
    ngs = [ng for ng in ngrams(tokens, n)]
    return ["_".join(ng) for ng in ngs if len(ng)>0 ]

## -- filter the existed tokens in a specific model
def get_existed_tokens(tokens, n_model):
    return [tok for tok in tokens if tok in n_model.wv ]

  

In [0]:
from gensim.models import KeyedVectors
from gensim.models import Word2Vec

t_model = gensim.models.Word2Vec.load('full_grams_cbow_300_twitter.mdl')
#model = KeyedVectors.load_word2vec_format('full_grams_cbow_300_twitter', binary=True)



In [11]:
# python 3.X
token = clean_str(u'ابو تريكه').replace(" ", "_")
if token in t_model.wv:
    most_similar = t_model.wv.most_similar( token, topn=10 )
    for term, score in most_similar:
        term = clean_str(term).replace(" ", "_")
        if term != token:
            print(term, score)

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


ابوتريكه 0.879421591758728
تريكه 0.7756417989730835
حازم_امام 0.7520479559898376
حسام_غالي 0.7310762405395508
وائل_جمعه 0.7296881675720215
حسام_حسن 0.7109734416007996
باسم_مرسي 0.7088809013366699
عمرو_زكي 0.6989202499389648
ميدو 0.6951170563697815
رمضان_صبحي 0.6935876607894897


In [12]:
pos_tokens=[ clean_str(t.strip()).replace(" ", "_") for t in ['عمرو دياب', 'الخليج'] if t.strip() != ""]
neg_tokens=[ clean_str(t.strip()).replace(" ", "_") for t in ['مصر'] if t.strip() != ""]

vec = calc_vec(pos_tokens=pos_tokens, neg_tokens=neg_tokens, n_model=t_model, dim=t_model.vector_size)

most_sims = t_model.wv.similar_by_vector(vec, topn=10)
for term, score in most_sims:
    if term not in pos_tokens+neg_tokens:
        print(term, score)


ماجد_المهندس 0.5671201944351196
عبدالله_رويشد 0.5305584669113159
عبدالمجيد_عبدالله 0.525513231754303
محمد_عبده 0.5064847469329834
راشد_الماجد 0.5064665675163269
رابح_صقر 0.5000637769699097
نبيل_شعيل 0.4981035590171814
الرويشد 0.4958586096763611
الجسمي 0.4870912432670593


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [34]:
# python 3.X
token = clean_str(u'تونس')
# python 2.7
# token = clean_str('تونس'.decode('utf8', errors='ignore'))
most_similar = t_model.wv.most_similar( token, topn=10 )
for term, score in most_similar:
    print(term, score)


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


ليبيا 0.775361180305481
الجزائر 0.7714784145355225
مصر 0.7435071468353271
فرنسا 0.70225989818573
موريتانيا 0.6786443591117859
تركيا 0.6531816124916077
طرابلس 0.6514291763305664
لبنان 0.6502016186714172
السودان 0.6472519636154175
الاردن 0.6358590126037598


In [28]:
# get a word vector
word_vector = t_model.wv[ token ]
print(word_vector)

[ 5.2427763e-01 -1.7721991e-01 -6.7358071e-01  5.5230027e-01
 -5.1873112e-01 -1.8911989e-01  7.9052705e-01 -1.4380398e+00
 -2.3931246e-01  7.0309275e-01 -3.6220643e-01  2.7617165e-01
 -1.8557517e-01 -9.4612902e-01 -1.7434624e+00 -1.2418944e+00
  1.5038666e+00  1.7104491e+00  3.0561200e-01  1.2476454e+00
  1.2952463e-02  2.7332184e-01  5.5270445e-01 -4.4741083e-02
  7.3687658e-02 -2.7608218e-02  2.9314157e-01 -3.2719314e-02
 -8.8603097e-01  5.3171492e-01 -1.9288963e-01  1.3752570e+00
  9.7422302e-01 -5.4750699e-01 -1.9364828e+00  1.7958424e+00
 -2.8463033e-01 -8.6037177e-01  9.8598814e-01 -2.0660942e+00
  1.4259132e-02 -5.3225791e-01 -2.4116476e-01 -2.1537793e-01
 -7.3768771e-01  1.4348507e+00  1.6797736e+00 -1.5312676e+00
 -1.4269476e-01 -7.2796893e-01 -1.4941980e+00  9.8120403e-01
  1.4845023e+00  1.9159906e+00 -2.1403081e+00  4.6480992e-01
 -2.4597912e+00 -8.1003863e-01 -4.4870052e-01 -2.6394494e+00
  3.3719936e-01  4.5325339e-01  1.1879311e+00 -2.4306817e+00
  2.9790580e-01  6.49341